In [1]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.8 MB/s eta 0:00:00


In [2]:
pip install qiskit_ibm_runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [3]:
pip install pylatexenc


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=56b3827ecf26611f2171706aad188d1d2b67d832b16e0d2c78bc19b291657dde
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [24]:
pip install qiskit-ibm-provider qiskit-aer


In [5]:
import qiskit

In [6]:
qiskit.__version__

'1.2.0'

In [7]:
from qiskit_ibm_runtime import QiskitRuntimeService


In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel="ibm_quantum", token='e6a0173341596e8c71785e3e9db1894b8c58f573417687f1e079d20d4015f25c81844c669d0a2b4957f364c14ee9ff71676154cee995f184c3142031e27ef14b')
backend = service.get_backend('ibm_brisbane')


<ipython-input-8-5c4a7926ea82>:3: DeprecationWarning: The get_backend() method is deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use backend() instead.
  backend = service.get_backend('ibm_brisbane')


In [9]:
import os
from qiskit_aer import Aer  # Use this if you need local simulation
import uuid
import pickle

def _prep():
    if 'archive' not in os.listdir():
        os.mkdir('archive')
_prep()



In [10]:
class Archive():
    '''
    A serializable equivalent to the Qiskit job object.
    '''
    def __init__(self, job, path='', note='', circuits=None):

        if 'job_id' in dir(job):
            job_id = job.job_id()
        else:
            job_id = uuid.uuid4().hex

        if 'credentials' in dir(job.backend().provider()):
            creds = job.backend().provider().credentials
            provider = creds.hub +'_'+ creds.group +'_'+ creds.project
        else:
            provider = str(job.backend().provider())

        self.archive_id = job_id + '@' + job.backend().name() + '@' + provider

        self.path = path

        self.note = note
        self._job_id = job.job_id()
        self._backend = job.backend()
        self._backend.properties() # just needs to be called to load
        self._metadata = job.metadata
        self.version = job.version
        if 'circuits' in dir(job):
            self._circuits = job.circuits()
        else:
            self._circuits = circuits
        if 'qobj' in dir(job):
            self._qobj = job.qobj()
        if 'aer' in self.backend().name() or 'simulator' in self.backend().name():
            self._result = job.result()
        else:
            self._result = None

        self.save()

    def save(self):
        with open(self.path + 'archive/'+self.archive_id, 'wb') as file:
            pickle.dump(self, file)

    def job_id(self):
        return self._job_id

    def backend(self):
        return self._backend

    def metadata(self):
        return self._job_id

    def circuits(self):
        return self._circuits

    def qobj(self):
        return self._qobj

    def result(self):
        if self._result==None:
            try:
                creds = self.backend().provider().credentials
                provider = creds.hub +'_'+ creds.group +'_'+ creds.project
                backend = get_backend(self.backend().name(), provider=provider)
            except:
                backend = get_backend(self.backend().name())
            job = backend.retrieve_job(self.job_id())
            self._result = job.result()
            self.save()
        return self._result


def get_backend(backend_name, provider=None):
    '''
    Given a string that specifies a backend, returns the backend object
    '''
    if type(backend_name) is str:
        if 'aer' in backend_name:
            backend = Aer.get_backend(backend_name)
        else:
            if provider==None:
                providers = IBMQ.providers()
            else:
                hub, group, project = provider.split('_')
                providers = [IBMQ.get_provider(hub, group, project)]
            no_backend = True
            for provider in providers:
                if no_backend:
                    backends = provider.backends()
                    for potential_backend in backends:
                        if potential_backend.name()==backend_name:
                            backend = potential_backend
                            no_backend = False
            if no_backend:
                print('No backend was found matching '+backend_name+' with your providers.')
    else:
        backend = backend_name
    return backend


def submit_job(circuits, backend_name, path='', note='',
               job_name=None, job_share_level=None, job_tags=None, experiment_id=None, header=None,
               shots=8192, memory=None, qubit_lo_freq=None, meas_lo_freq=None, schedule_los=None,
               meas_level=None, meas_return=None, memory_slots=None, memory_slot_size=None,
               rep_time=None, rep_delay=None, init_qubits=None, parameter_binds=None, use_measure_esp=None,
               noise_model=None, **run_config):
    '''
    Given a backend name and the arguments for the `run` method of the backend object, submits the job
    and returns the archive id.
    '''

    # get backend
    if type(backend_name) is str:
        backend = get_backend(backend_name)
    else:
        backend = backend_name

    # submit job
    if 'fake' not in backend.name():
        job = backend.run(circuits, job_name=job_name, job_share_level=job_share_level, job_tags=job_tags,
                          experiment_id=experiment_id, header=header, shots=shots, memory=memory,
                          qubit_lo_freq=qubit_lo_freq, meas_lo_freq=meas_lo_freq, schedule_los=schedule_los,
                          meas_level=meas_level, meas_return=meas_return, memory_slots=memory_slots,
                          memory_slot_size=memory_slot_size, rep_time=rep_time, rep_delay=rep_delay, init_qubits=init_qubits,
                          parameter_binds=parameter_binds, use_measure_esp=use_measure_esp, noise_model=noise_model,
                          **run_config)
    else:
        job = backend.run(circuits, job_name=job_name, job_share_level=job_share_level, job_tags=job_tags,
                          experiment_id=experiment_id, header=header, shots=shots, memory=memory,
                          qubit_lo_freq=qubit_lo_freq, meas_lo_freq=meas_lo_freq, schedule_los=schedule_los,
                          meas_level=meas_level, meas_return=meas_return, memory_slots=memory_slots,
                          memory_slot_size=memory_slot_size, rep_time=rep_time, rep_delay=rep_delay, init_qubits=init_qubits,
                          parameter_binds=parameter_binds, use_measure_esp=use_measure_esp,
                          **run_config)

    # create archive
    archive = Archive(job, note=note, circuits=circuits)

    # return the id
    return archive.archive_id


def get_job(archive_id):
    '''
    Returns the Qiskit job object corresponding to a given archive_id
    '''
    job_id, backend_name, provider = archive_id.split('@')
    backend = get_backend(backend_name, provider = provider)
    job = backend.retrieve_job(job_id)
    return job


def get_archive(archive_id, path=''):
    '''
    Returns the saved archive object corresponding to a given archive_id
    '''

    # create archive if file doesn't exist
    filename = path + 'archive/'+archive_id
    if archive_id not in os.listdir(path + 'archive/'):
        job_id, backend_name, provider = archive_id.split('@')
        new_archive_id = jobid2archive(job_id, backend_name, path=path, provider=provider)
        assert new_archive_id==archive_id

    with open(filename, 'rb') as file:
        archive = pickle.load(file)
    return archive


def jobid2archive(job_id, backend_name, path='', provider=None):

    backend = get_backend(backend_name, provider)
    job = backend.retrieve_job(job_id)

    archive = Archive(job, path=path)
    archive.result()

    return archive.archive_id

Benchamrk code


In [13]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# Step 2: Open a session with the backend
with Session(backend=backend) as session:

    # Step 3: Initialize the Estimator primitive within the session
    estimator = Estimator(mode=session)

    # Step 4: Enable dynamical decoupling to suppress coherent errors on idle qubits
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"

    # Step 7: Define a simple quantum circuit
    qc = QuantumCircuit(2)
    qc.h(1)
    qc.cx(0, 1)



    # Define an observable acting on both qubits, e.g., Z on the first qubit and I on the second
    observable = SparsePauliOp.from_list([("ZI", 1)])

    # Step 8: Transpile and run the circuit using the Estimator with the observable
    job = estimator.run([(qc, observable)])

    # Step 9: Retrieve and process the results
    result = job.result()
    print("Result:", result)

# Step 10: The session is automatically closed after exiting the 'with' block


IBMInputValueError: 'The instruction h on qubits (1,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'

In [19]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import XGate, YGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes.scheduling import ALAPScheduleAnalysis, PadDynamicalDecoupling
from qiskit.transpiler import PassManager
from qiskit.visualization import timeline_drawer
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel
from qiskit.transpiler.passes import BasisTranslator

# Step 2: Create a quantum circuit
qc = QuantumCircuit(2)
qc.h(1)
qc.cx(0, 1)

# Step 3: Define an observable acting on both qubits (Z on first qubit, I on second qubit)
observable = SparsePauliOp.from_list([("ZI", 1)])

# Step 4: Transpile the circuit using a pass manager for the backend
target = backend.target
basis_gates = list(target.operation_names)
pm = generate_preset_pass_manager(optimization_level=1, target=target, seed_transpiler=12345)
qc_transpiled = pm.run(qc)

# Step 5: Define a dynamical decoupling sequence (XY4 sequence)
X = XGate()
Y = YGate()
dd_sequence = [X, Y, X, Y]

# Step 6: Apply ALAPScheduleAnalysis and PadDynamicalDecoupling passes
dd_pm = PassManager([ALAPScheduleAnalysis(target=target), PadDynamicalDecoupling(target=target, dd_sequence=dd_sequence)])
qc_dd = dd_pm.run(qc_transpiled)

# Step 7: Apply BasisTranslator to remove YGates and use only basis gates
qc_dd = BasisTranslator(sel, basis_gates)(qc_dd)

# Step 8: Transform the observable using the transpiled circuit's layout
observable_isa = observable.apply_layout(layout=qc_transpiled.layout)

# (Optional) Visualize the dynamical decoupling and delays
timeline_drawer(qc_dd, show_idle=False)

# Step 9: Open a session with the backend and run the Estimator with dynamical decoupling
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    # Enable dynamical decoupling and choose sequence type
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"

    # Run the transpiled circuit with the layout-applied observable
    job = estimator.run([(qc_dd, observable_isa)])

    # Step 10: Retrieve and process the results
    result = job.result()
    print("Result:", result)


Result: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': True, 'sequence_type': 'XpXm', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})


In [20]:
expectation_value = result[0].data.evs
print("Expectation Value:", expectation_value)


Expectation Value: 0.026804123711340205


In [21]:
standard_deviation = result[0].data.stds
ensemble_error = result[0].data.ensemble_standard_error

print("Standard Deviation:", standard_deviation)
print("Ensemble Standard Error:", ensemble_error)


Standard Deviation: 0.018682641759834348
Ensemble Standard Error: 0.016490744814700347


closer to Whooten paper...

In [35]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, YGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes.scheduling import ALAPScheduleAnalysis, PadDynamicalDecoupling
from qiskit.transpiler import PassManager
from qiskit.visualization import timeline_drawer
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel
from qiskit.transpiler.passes import BasisTranslator
from qiskit.quantum_info import SparsePauliOp

# Step 2: Create a quantum circuit (Bell State Example)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Step 3: Define an observable acting on both qubits (Z on first qubit, I on second qubit)
observable = SparsePauliOp.from_list([("ZI", 1)])

target = backend.target
basis_gates = list(target.operation_names)

# Pass manager for transpiling with the backend
pm = generate_preset_pass_manager(optimization_level=1, target=target, seed_transpiler=12345)
qc_transpiled = pm.run(qc)

# Step 5: Define a dynamical decoupling sequence (XY4 sequence)
X = XGate()
Y = YGate()
dd_sequence = [X, Y, X, Y]

# Step 6: Apply ALAPScheduleAnalysis and PadDynamicalDecoupling passes
dd_pm = PassManager([ALAPScheduleAnalysis(target=target), PadDynamicalDecoupling(target=target, dd_sequence=dd_sequence)])
qc_dd = dd_pm.run(qc_transpiled)

# Step 7: Apply BasisTranslator to remove YGates and use only basis gates
qc_dd = BasisTranslator(sel, basis_gates)(qc_dd)

# Step 8: Transform the observable using the transpiled circuit's layout
observable_isa = observable.apply_layout(layout=qc_transpiled.layout)

# (Optional) Visualize the dynamical decoupling and delays
timeline_drawer(qc_dd, show_idle=False)

# Step 9: Open a session with the backend and run the Estimator with dynamical decoupling
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    # Enable dynamical decoupling and set the sequence type
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"

    # Run the transpiled circuit with the layout-applied observable
    job = estimator.run([(qc_dd, observable_isa)])

    # Step 10: Retrieve and process the results
    result = job.result()
    print("Full Result Object:", result)



Full Result Object: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': True, 'sequence_type': 'XpXm', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})


In [36]:
# Step 10: Retrieve and process the results
result = job.result()

# Access the expectation value
expectation_value = result[0].data.evs
print("Expectation Value:", expectation_value)


Expectation Value: 0.042012448132780086
